# Cross-Dataset Comparison

Comparing model performance across different datasets to understand generalization.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="darkgrid")
%matplotlib inline

## Summary

| Dataset | Accuracy | Precision | Recall | F1 |
|---------|----------|-----------|--------|----|
| Original Eval | 99.7% | - | - | - |
| In-The-Wild | 42.0% | 30.4% | 12.4% | 17.6% |
| Fake-or-Real | TODO | TODO | TODO | TODO |

In [ ]:
# comparison data
results = {
    "dataset": ["Original Eval", "In-The-Wild", "Fake-or-Real"],
    "accuracy": [99.7, 42.0, None],  # TODO: fill in
    "f1": [None, 17.6, None]
}

df = pd.DataFrame(results)
df

In [ ]:
# TODO: visualization comparing datasets
# fig, ax = plt.subplots(figsize=(8, 5))
# ax.bar(df["dataset"], df["accuracy"])
# ax.set_ylabel("Accuracy (%)")
# ax.set_title("Model Performance Across Datasets")

## Conclusions

TODO: analysis after running on both datasets

Key questions:
- Does the model generalize poorly in general, or is In-The-Wild particularly challenging?
- What characteristics make a dataset harder for the model?
- Is the issue with the model architecture or training data?